# 第 7 章 アンサンブル

## 7.1 アンサンブルとは
- 複数のモデルを組み合わせてモデルを作ること・予測すること
- 分析コンペだとふつうモデルのアンサンブルによる予測値を提出する。
    - 数百のモデルを組み合わせることさえある
- 実務では少し精度を上げる程度の目的でモデルを複数作るのは許されないこともあるかもしれない
    - 検討・計算に膨大な時間がかかる
    - コンペはまた別
    - チームを組むときにそれぞれの成果を混ぜ合わせることもある
- 2 つの主な手法
    - 平均を取るようなシンプルなアンサンブル
    - 効率的にモデルを混ぜ合わせる手法：**スタッキング**

## 7.2 シンプルなアンサンブル手法

### 7.2.1 平均・加重平均

- 回帰タスク：単に複数のモデルの予測値の平均を取る
    - これで十分精度が出ることもある
- タスクやモデルによって考えるべき要素は次の通り
    - ハイパーパラメータや特徴量が同じでも、乱数シードを変えて平均を取るだけで精度が上がることもある
    - ニューラルネットワークは特に学習の精度がぶれやすく、効果が出やすい

#### たくさん作ったモデルの精度にばらつきがある場合
- 精度の高いモデルには大きめの重みをかけた加重平均をしたくなる
- どうやって重みを決めるか？
    - モデルの精度を見ながら**適当に決める**
    - スコアがもっとも高くなるように最適化する
        - 最適化には `scipy.optimize` モジュールが使える

#### 注意点
- 「2.5.3 閾値の最適化を out-of-fold で行うべきか」も参照
- 学習データ全体の予測値を使い、学習データ全体のスコアが最適になるように調整したとき、目的変数を知っている状態での調整になる
    - 少し過大評価になってしまう
    - 避けたければクロスバリデーションして out-of-fold
    - スタッキングで 2 層目に線型モデルを使うのと類似

#### 復習：out-of-fold
- cf. P94
- データをいくつかに分割し、そのうち 1 つを予測対象にし、残りを予測用の学習データにする手法
- 場面の設定
    - 分析コンペである変数を予測し、その予測した結果を使いたい・予測が正しいか評価したい
    - 各レコードについて自身の変数の値をつかってしまうと答えを見ているようなもので予測の意味をなさない
    - 自身の変数の値を使わずに予測したい
- よく使われるのはクロスバリデーション：cf 5.2.2

### 7.2.2 多数決・重みづけ多数決
- 分類タスクの場合
    - 一番シンプルなのは予測値のクラスの多数決
    - モデルごとに重みをつけて多数決を取ることもある
- ふつうは予測確率をもとに予測値のクラスを決めているはず
    - 予測値のクラスよりも情報の多い予測確率が使える
    - 予測確率の平均・重みづけ平均を取った後に分類する方法もある

### 7.2.3 注意点とその他のテクニック

#### 評価指標の最適化
- cf. 「2.5 評価指標の最適化」
- 評価指標によってはモデルの予測値をそのまま提出するのではなく、評価指標に合わせるために最適化する必要がある。
- アンサンブルの前に個別のモデルの予測値に対して最適化するかどうかは状況次第
    - アンサンブルの後に最適化する必要があるケースは多い

#### 不思議な調整
- 「理屈」のない調整もありうる
- 「試行錯誤する中でたまたまスコアがよくなったから採用」というケース

#### 順位の平均をとる
- AUC のような予測値の大小関係だけが影響する評価指標を考える
- 確率の平均値ではなく確率を順位に変換して順位の平均値をとる
    - モデルが予測する確率がゆがんでいてもその影響を除いてアンサンブルできる。
    - cf. P95、確率の歪み

#### 幾何平均や調和平均などの利用
- 算術平均（標本平均）以外の平均を使う
- 幾何平均・調和平均・p次平均
- cf. P359 図7.1：平均とその等高線の図

#### 過学習気味のモデルのアンサンブル
- 「複雑でやや過学習気味のモデルを選ぶ方がいい」という意見もある
    - 言葉の定義
        - バイアス：モデルの平均的な予測値と真の値の乖離
        - バリアンス：予測値の不安定性
    - モデルが複雑：バイアス小、バリアンス大
    - モデルが単純：バイアス大、バリアンス小
    - アンサンブルで複数の予測値を組み合わせるとバリアンスは小さくなる。
    - 「少し複雑なモデルにしておけばトータルのバイアスを抑えられるのでは？」

## 7.3 スタッキング


### 7.3.1 スタッキングの概要
- cf. P.361 図7.2-7.4
- 効率的・効果的に 2 つ以上のモデルを組み合わせて予測する方法
- 次の手順で進める
    - 学習データをクロスバリデーションの fold にわける
        - fold を 1-4 とする
    - モデルを out-of-fold で学習させ、バリデーションデータへの予測値を作る
        - fold2-fold4 で学習したモデルで fold1 の予測値を作る
            - これを fold 分繰り返してから予測値をもとの順番に並べ直す
        - 学習データに特徴量として「そのモデルでの予測値」を作る
    - 各 fold で学習したモデルでテストデータを予測し、平均などを取ってテストデータの特徴量とする
    - 直上の 2 ステップをスタッキングしたいモデルの数だけくり返す：図 7.3
    - 直上の 3 ステップで作った特徴量でモデルの学習・予測をする。
        - このモデルを 2 層目のモデルという

#### コメント
- ポイント：こうして作った特徴量は予測対象のレコードの目的変数を知らない状況で学習したモデルによる予測値であること
- 問題があるパターン
    1. 学習データをクロスバリデーションの fold にわけず、 fold1-fold4 まですべてを学習データにしたモデルで学習データをそのまま予測する
    2. 上記モデルでテストデータを予測する
    3. 上記 2 ステップをスタッキングしたいモデルの数だけくり返す
    4. 2 層目のモデルでは 1-2 で作った予測値を特徴量としてモデルの学習・予測をする
- この方法の問題
    - 学習データは「目的変数を知っている」予測値
    - テストデータについて「目的変数を知らない」予測値
    - 学習データとテストデータで意味が違う特徴量になっている
    - 2 層目のモデルでテストデータを予測すると精度が悪くなる
        - cf. P362, 図 7.5

### 7.3.2 特徴量作成の方法としてのスタッキング
- スタッキングはアンサンブル手法
- 特徴量を作る手法ととらえることもできる
- スタッキングで作った値は「あるモデルによる予測値」という特徴量ともみなせる：**メタ特徴量**
- 特徴量とみなすとき
    - 同質性が重要: 「あるモデルの予測値」という特徴量が学習データに対してもテストデータに対しても同じ意味の特徴量であること
    - cf. 先程の問題のケース
- 他の問題があるとき: 一部のモデルで作られた予測値が「目的変数を少し知っている」
    - target encoding の適用に間違いがあったとき
        - target encoding の復習：目的変数を使ってカテゴリ変数を数値に変換すること
    - パラメータチューニングしすぎたとき
- 「あるモデルによる予測値」とみなすと工夫の幅が広がる
    - 普通のアプローチ：目的変数を予測するモデルを作る
    - 次のようなアプローチが取れるようになる
        - モデルのとらえ直し
            - 欠損が多い変数の値を予測するモデル
            - 回帰問題を目的変数の値が 0 か否かの 2 値分類問題とみなす
        - それらの予測値を特徴量にする
    - 2 層目のモデルにスタッキングで作った特徴量と一緒に元のデータの特徴量や t-SNE などの教師なし学習による特徴量を与えることなどもある

### 7.3.3 スタッキングの実装


In [0]:
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold

# models.pyにModel1Xgb, Model1NN, Model2Linearを定義しているものとする
# 各クラスは、fitで学習し、predictで予測値の確率を出力する

from models import Model1Xgb, Model1NN, Model2Linear


# 学習データに対する「目的変数を知らない」予測値と、テストデータに対する予測値を返す関数
def predict_cv(model, train_x, train_y, test_x):
    preds = []
    preds_test = []
    va_idxes = []

    kf = KFold(n_splits=4, shuffle=True, random_state=71)

    # クロスバリデーションで学習・予測を行い、予測値とインデックスを保存する
    for i, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        model.fit(tr_x, tr_y, va_x, va_y)
        pred = model.predict(va_x)
        preds.append(pred)
        pred_test = model.predict(test_x)
        preds_test.append(pred_test)
        va_idxes.append(va_idx)

    # バリデーションデータに対する予測値を連結し、その後元の順序に並べ直す
    va_idxes = np.concatenate(va_idxes)
    preds = np.concatenate(preds, axis=0)
    order = np.argsort(va_idxes)
    pred_train = preds[order]

    # テストデータに対する予測値の平均をとる
    preds_test = np.mean(preds_test, axis=0)

    return pred_train, preds_test


# 1層目のモデル
# pred_train_1a, pred_train_1bは、学習データのクロスバリデーションでの予測値
# pred_test_1a, pred_test_1bは、テストデータの予測値
model_1a = Model1Xgb()
pred_train_1a, pred_test_1a = predict_cv(model_1a, train_x, train_y, test_x)

model_1b = Model1NN()
pred_train_1b, pred_test_1b = predict_cv(model_1b, train_x_nn, train_y, test_x_nn)

# 1層目のモデルの評価
print(f'logloss: {log_loss(train_y, pred_train_1a, eps=1e-7):.4f}')
print(f'logloss: {log_loss(train_y, pred_train_1b, eps=1e-7):.4f}')

# 予測値を特徴量としてデータフレームを作成
train_x_2 = pd.DataFrame({'pred_1a': pred_train_1a, 'pred_1b': pred_train_1b})
test_x_2 = pd.DataFrame({'pred_1a': pred_test_1a, 'pred_1b': pred_test_1b})

# 2層目のモデル
# pred_train_2は、2層目のモデルの学習データのクロスバリデーションでの予測値
# pred_test_2は、2層目のモデルのテストデータの予測値
model_2 = Model2Linear()
pred_train_2, pred_test_2 = predict_cv(model_2, train_x_2, train_y, test_x_2)
print(f'logloss: {log_loss(train_y, pred_train_2, eps=1e-7):.4f}')

### 7.3.4 スタッキングのポイント

#### スタッキングが効く場合・効かない場合
- コンペの性質によってスタッキングの効果も違う
- スタッキングは学習データを使い尽くそうとする性質がある
    - 学習データとテストデータが同じ分布でデータ量が多いと有効
    - 時系列データのような分布が違うデータは学習データに適合しすぎる
        - スタッキングよりもモデルの加重平均によるアンサンブルを使う方が多い
- 特徴量作成で差がつきにくい場合は相対的に有効
- 評価指標の違い
    - logloss を使うときスタッキングが有効
        - accuracy よりも logloss の方が細かく予測値をチューニングすることによるスコアの向上がある
    - 多クラス分類で評価指標が multi-class logloss の場合
        - GBDT とニューラルネットをスタッキングすると大きなスコア向上がありうる

#### テストデータの特徴量の作成方法
- スタッキングでテストデータの特徴量を作るときはテストデータへの予測が必要
- 予測の方法について
    - ここまでは P.366 図 7.6 のような各 fold のモデル平均として説明した
    - 図 7.7 のように学習データ全体に対して学習し直したモデルで予測する方法もある
- クロスバリデーション後にテストデータをどう予測するかはいつでも問題
    - cf. P218, 4.1.2 モデル作成の流れ

#### 2 層目のモデルに元の特徴量を加えるか?
- 1 層目のモデルの予測値だけを特徴量にするか, 1 層目のモデルの元の特徴量も付加するか?
    - cf. P367, 図 7.8
    - 前者 (予測値だけ): 学習時間が少なく過学習も起きにくい
    - 後者: 元の特徴量とモデルの予測値の関係性が見える
        - t-SNE, UMAP やクラスタリングなどの教師なし学習による特徴量を 2 層目のモデルの特徴量に与えることもある

#### 多層のスタッキング
- イメージ図：P.367, 図 7.9
- スタッキングをくり返すと精度の上がり方は弱くなる
    - それでも多少の効果はある
- スタッキングの選択肢があるとき
    - 2 層目で両方試して 3 層目で組み合わせるとか


#### ハイパーパラメータ調整やアーリーストッピングでの注意点
- 調整しすぎるとバリデーションデータへの過剰適合が起きる
- アーリーストッピングではバリデーションデータに対して学習の進行が最適なところで止まる
- スタッキング: 特徴量として予測値を使う
    - バリデーションに対しては「少しだけ目的変数を知っている」
    - テストデータに対してはそうではない
- ハイパーパラメータ調整, アーリーストッピングでのパラメータ・決定木の本数決定のあとに
  fold の切り方を変えて学習・予測した方がいい?
    - cf. 5.4.5 バリデーションデータや Public Leaderboard への過剰な適合


#### 最終的に出力すべき予測値でなくても良い
- 1 層目のモデルで出力する値は予測に役に立てばいい
    - 最終出力する予測値である必要はない
- いろいろな工夫がありうる

#### モデルの予測値のさらなるメタ特徴量
- 2 層目のモデルに与える特徴量を考える
    - 1 層目のモデルの予測値
- メタ特徴量を考えてもいい
    - 1 層目のあるモデルと別のモデルによる予測値の差
    - 1 層目の複数のモデルの予測値の平均や分散

#### 分析への利用
- スタッキングでモデルの予測値という特徴量が手に入る
- 目的変数と組み合せるとレコードごとにどの程度正しく予測できているかわかる
    - これを分析に応用する
- 例: 次のような量で精度を見て予測が難しいレコードの条件を推察する
    - 混同行列 (分析タスクで真の値のクラスと予測値のクラスの行列) 作成
    - あるカテゴリ変数の値

### 7.3.5 hold-out データへの予測値を用いたアンサンブル
- Blending というテクニック
    - 予測値の加重平均によるアンサンブル
    - ここでは特に「hold-out データへの予測値によるアンサンブル」とする
    - 復習（hold-out 法）
        - 一部のデータをバリデーションデータとして取り分けておく
        - 欠点：モデルの学習・評価に使えるデータが減る
        - 欠点克服のためにふつうクロスバリデーションをよく使う
- モデルの予測値を次の層の特徴量として使う
- 相違点: まず hold--out データを分ける
    - スタッキング: クロスバリデーションの分割ごとに学習させる
- 2 層目では hold-out データで学習し, テストデータを予測する
- 手順：cf. P.370, 図 7.11
    - 学習データを train データと hold-out データにわける
    - モデルを train データで学習させ, hold-out データ・テストデータの予測値を作る
        - cf. P.370, 図 7.10
    - 上記プロセスをアンサンブルしたいモデルの数だけくり返す
    - 2 層目のモデルとして直上 2 ステップで作った特徴量でモデルを学習・予測する
- メリット
    - 1 層目のモデルをクロスバリデーションしないので計算時間が短い
    - 1 層目のモデルの学習時に hold-out データの目的変数を見ないためリークのリスクが (少し) 小さい
- デメリット
    - 使えるデータ数が少なくなる: かなり強い否定的な材料
- データ数が多く, クロスバリデーションするための計算量が厳しいときに検討する


## 7.4 どんなモデルをアンサンブルにすると良いか？
- アンサンブルで高い効果を出すためには多様性に富んだモデルを組み合わせる
    - 得意な部分が違うモデルを組み合わせる
    - 例: それぞれ晴れの日の販売量と雨の日の販売量をよく予測できるモデル
    - 例: 線型関係をよく捉えるモデルと変数間の相互作用を良く捉えるモデル
- 精度が低いモデルでも, 性質が違うならアンサンブルとしては精度の改善につながることがある


### 7.4.1 多様なモデルを使う
- 予測値の境界が違うため, 互いの弱いところを補完する
    - まずは単体で精度が高い GBDT とニューラルネットでのアンサンブルを試してみよう
    - GBDT
    - ニューラルネット
    - 線型モデル
    - k 近傍法
    - Extremely Randomized Trees (ERT) またはランダムフォレスト
    - Regularized Greedy Forest (RGF)
    - Field-aware Factorization Machines (FFM)


#### Infomation
- 良いスタッキングのソリューションの特徴
    - 2-3 つの GBDT: 決定木の深さの大中小
    - 1-2 つのランダムフォレスト: 決定木の深さの大小
    - 1-2 つのニューラルネット: 層の数の大小
    - 1 つの線型モデル

### 7.4.2 ハイパーパラメータを変える
- モデルが同じでもハイパーパラメータを変えてみるのも一手



### 7.4.3 特徴量を変える
- 使う特徴量とその組み合わせを変える
    - ある特徴量の組を使うか使わないか
    - 特徴量のスケーリングをするかしないか
    - 特徴選択を強くするかしないか
    - 外れ値を除くか除かないか
    - データの前処理や変換の方法を変える

### 7.4.4 問題のとらえ方を変える
- 問題の捉え方を変えたり, 問題を解く助けになる何らかの値を予測するモデルを作り,
  その予測値を特徴量にする
    - 回帰タスクである値以上・以下の二値分類タスクのモデルを作る
    - 0 以上の値を取る販売額のタスクで販売されたかどうかの二値分類タスクのモデルを作る
    - 多クラス分類で一部のクラスだけを予測するモデルを作り,
      そのモデルではその一部のクラスに特化した手法を使う
    - 重要だが欠損が多い特徴量を予測するモデルを作る
    - あるモデルによる予測値の残差 (= 目的変数-予測値) を予測するモデルを作る


### 7.4.5 スタッキングに含めるモデルの選択
- スタッキングに含めるモデルの選択について確立した方法はない
- 単純な手法
    - モデルを作るごとにスタッキングのモデルに含める
        - 精度がよくなれば残し, そうでなければ対象外
        - これをくり返す
    - 自動化手法としては「6.2.3 反復して探索する手法」での Greedy Forward Selection
    - 計算量によってはこれも難しい
- 他の方法
    - 相関係数が 0.95 以下, コルモゴロフ-スミルノフ検定統計量が 0.05 以上のモデルを精度が高い順に選ぶ
    - 単に精度が高いモデルを選ぶだけだと同じようなモデルばかりで多様性がなくなることに配慮
- モデル選択の上での補助的な分析手法
    - バリデーションの結果をログに出し, 各モデルのスコアを把握する
    - 多様性評価
        - 予測値の相関係数を計算する
        - 異なるモデルの予測値同士の散布図をプロットする
    - バリデーションスコアとモデルの予測値を単独で提出したときの Public Leaderboard のスコアをプロットする



#### AUTHOR'S OPINION
- アンサンブルによるソリューションの意義の議論
    - 数百個のモデルをアンサンブルしたモデルで少し精度が上がったとして, 何か意味があるか?
- 例えば次のような意義がある
    - スタッキングの手法は複数のモデルを混ぜ合わせる効果的かつシンプルな方法
    - 実務的には, タスクによっては少しの精度向上が本質的に重要で多くの利益をもたらすことがある
    - アンサンブルで達成した精度とシンプルなアプローチでの精度を比較できることにも意義がある


## 7.5 分析コンペにおけるアンサンブルの例

### 7.5.1 Kaggle の「Otto Group Product Classification Challenge
- 匿名化された特徴量をもとに商品を 9 クラスの商品カテゴリに分類する多クラス分類タスク
- 評価指標が multi-class logloss
- モデルについては P.375 参照
    - GBDT・ニューラルネット・k 近傍法をはじめおいた多数のモデルのアンサンブル
- 2 位も P.377 図 7.12 のようなスタッキングを使っている

### 7.5.2 Kaggle の「Home Depot Product Search Relevance」
- Home Depot のサイトで検索された語句と商品の関連度を予測する
- 評価指標は RMSE (平均 2 乗誤差)
- 検索された語句や商品のタイトル・説明がテキストで提供
    - 自然言語処理の技術が問われる
- 3 位のソリューション
    - テキストに対する前処理・さまざまな特徴量作成が前提
    - GBDT・ニューラルネット・線型モデルによるスタッキング: P.378, 図 7.13
    - コードとドキュメントも公開されている

### 7.5.3 Kaggle の「Home Credit Default Risk」
- 消費者金融での顧客の貸し倒れ率の予測
- 評価指標は AUC
- 学習データとテストデータは時系列とプロジェクトで分割
    - プロジェクトはサービス開始地域や商品性のこと
- 分割特性による問題
    - クロスバリデーションによる学習データの評価値と Public Leaderboard でのスコアの整合性を取るのが難しい
    - スタッキングすると過学習する傾向にあった
- 2 位のソリューション
    - 独自手法を使っている
    - 「5.4.3 学習データとテストデータの分布が違う場合」で出てきた adversarial validation を利用した手法
    - 加重平均を取ってアンサンブル
    - 学習データではなくテストデータにあうように各モデルの重みを調整する
        - テストデータに近い学習データをサンプリングして使う
    - 手順
        - 学習データとテストデータに対して adversarial validation する
            - 学習データに対する「テストデータらしさ」を予測するモデルを作る
        - 各モデルでの予測値を out-of-fold で求める
        - 「テストデータらしさ」をもとに学習データの中から一定の割合でデータをサンプリング
        - サンプリングしたデータに対して加重平均の各モデルの重みを最適化
        - 重みの平均値が収束するまでくり返す

In [0]:
# モデルの予測値を加重平均する重みの値をadversarial validationで求める
# train_x: 各モデルによる確率の予測値（実際には順位に変換したものを使用）
# train_y: 目的変数
# adv_train: 学習データのテストデータらしさを確率で表した値

from scipy.optimize import minimize
from sklearn.metrics import roc_auc_score

n_sampling = 50  # サンプリングの回数
frac_sampling = 0.5  # サンプリングで学習データから取り出す割合


def score(x, data_x, data_y):
    # 評価指標はAUCとする
    y_prob = data_x['model1'] * x + data_x['model2'] * (1 - x)
    return -roc_auc_score(data_y, y_prob)


# サンプリングにより加重平均の重みの値を求めることを繰り返す
results = []
for i in range(n_sampling):
    # サンプリングを行う
    seed = i
    idx = pd.Series(np.arange(len(train_y))).sample(frac=frac_sampling, replace=False,
                                                    random_state=seed, weights=adv_train)
    x_sample = train_x.iloc[idx]
    y_sample = train_y.iloc[idx]

    # サンプリングしたデータに対して、加重平均の重みの値を最適化により求める
    # 制約式を持たせるようにしたため、アルゴリズムはCOBYLAを選択
    init_x = np.array(0.5)
    constraints = (
        {'type': 'ineq', 'fun': lambda x: x},
        {'type': 'ineq', 'fun': lambda x: 1.0 - x},
    )
    result = minimize(score, x0=init_x,
                      args=(x_sample, y_sample),
                      constraints=constraints,
                      method='COBYLA')
    results.append((result.x, 1.0 - result.x))

# model1, model2の加重平均の重み
results = np.array(results)
w_model1, w_model2 = results.mean(axis=0)

#### 手法に対する注意
- 学習データとテストデータの性質が大きく違う場合の手法
    - このケースでは adversarial validation での AUC は 0.9 以上
- スコアへの寄与は特徴量の改善ほど大きくない
    - コンペ終盤でのスコアの最後の一押し程度